In [ ]:
# use magic command to auto-reload
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import igraph as ig
import json
from dotenv import load_dotenv

sys.path.insert(0, os.path.join(os.getcwd(), '..'))  # parent folder of src
from src.simulation import run_simulation
from src.save import save_simulation_results, show_discussion_from_saved, load_replica
from src.visualization import show_interactive_conversation_tree_with_agent_selection
from dotenv import dotenv_values
import os
import asyncio
import aiohttp
from metrics import calculate_all_metrics, print_summary

In [ ]:
run_path = "runs/Exp_W_post"
results = calculate_all_metrics(run_path)
print_summary(results)


run_path = "runs/Exp_W_personal"
results = calculate_all_metrics(run_path)
print_summary(results)

run_path = "runs/Exp_W_success"
results = calculate_all_metrics(run_path)
print_summary(results)

In [ ]:
def load_params(path="PARAMS.json"):
    with open(path, "r") as f:
        params = json.load(f)

    # normalize legacy keys
    if "warmup_length" not in params and "warmup_length" in params:
        params["warmup_length"] = params.pop("warmup_length")

    return params


# --- load base params ---
PARAMS = load_params("PARAMS.json")

# --- experiment knobs (same as you used before) ---
PARAMS["time_decay_rate"]    = np.log(2) / 2   # half-life = 2 timesteps
PARAMS["W_agent_success"]    = 1
PARAMS["W_personal_weights"] = 0
PARAMS["W_post_success"]     = 0
PARAMS["noise_level"]        = 0.01

# (optional) quick overrides
PARAMS["replicas"]      = 1
PARAMS["num_agents"]    = 30
PARAMS["timesteps"]     = 30
PARAMS["warmup_length"] = 3



# --- secrets from .env (kept out of JSON for safety) ---
ENV = dotenv_values(".env")
for k in ("HF_TOKEN", "API_URL", "MODEL"):
    if k not in ENV or not ENV[k]:
        raise RuntimeError(f"Missing {k} in .env")
    PARAMS[k] = ENV[k]

print(PARAMS)

In [11]:
# --- run simulation ---
(List_of_WEIGHTS,
    List_of_READ_MATRIX,
    List_of_LIKES,
    List_of_POSTS,
    List_of_GRAPHS,
    List_of_INDIVIDUAL_LIKES) = await run_simulation(PARAMS)

# --- save results (new signature incl. INDIVIDUAL_LIKES) ---
out_root = "runs/Exp_W_success_GEMMA"
save_simulation_results(
    out_root,
    List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES,
    List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES,
    PARAMS
)
print(f"Saved run to: {out_root}")

3 0
T_current: 3
4 0
T_current: 4
5 0
T_current: 5
6 0
T_current: 6
7 0
T_current: 7
8 0
T_current: 8
9 0
T_current: 9
10 0
T_current: 10
11 0
T_current: 11
12 0
T_current: 12
13 0
T_current: 13
14 0
T_current: 14
15 0
T_current: 15
16 0
T_current: 16
17 0
T_current: 17
18 0
T_current: 18
19 0
T_current: 19
20 0
T_current: 20
21 0
T_current: 21
22 0
T_current: 22
23 0
T_current: 23
24 0
T_current: 24
25 0
T_current: 25
26 0
T_current: 26
27 0
T_current: 27
28 0
T_current: 28
29 0
T_current: 29
30 0
T_current: 30
31 0
T_current: 31
32 0
T_current: 32
Saved run to: runs/Exp_W_success_GEMMA


In [ ]:
# --- run simulation ---
(List_of_WEIGHTS,
    List_of_READ_MATRIX,
    List_of_LIKES,
    List_of_POSTS,
    List_of_GRAPHS,
    List_of_INDIVIDUAL_LIKES) = await run_simulation(PARAMS)

# --- save results (new signature incl. INDIVIDUAL_LIKES) ---
out_root = "runs/Exp_W_success_1"
save_simulation_results(
    out_root,
    List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES,
    List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES,
    PARAMS
)
print(f"Saved run to: {out_root}")

In [ ]:
# --- run simulation ---
(List_of_WEIGHTS,
    List_of_READ_MATRIX,
    List_of_LIKES,
    List_of_POSTS,
    List_of_GRAPHS,
    List_of_INDIVIDUAL_LIKES) = await run_simulation(PARAMS)

# --- save results (new signature incl. INDIVIDUAL_LIKES) ---
out_root = "runs/Exp_W_success_2"
save_simulation_results(
    out_root,
    List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES,
    List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES,
    PARAMS
)
print(f"Saved run to: {out_root}")

In [ ]:
# --- run simulation ---
(List_of_WEIGHTS,
    List_of_READ_MATRIX,
    List_of_LIKES,
    List_of_POSTS,
    List_of_GRAPHS,
    List_of_INDIVIDUAL_LIKES) = await run_simulation(PARAMS)

# --- save results (new signature incl. INDIVIDUAL_LIKES) ---
out_root = "runs/Exp_W_success_3"
save_simulation_results(
    out_root,
    List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES,
    List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES,
    PARAMS
)
print(f"Saved run to: {out_root}")

In [ ]:
# --- run simulation ---
(List_of_WEIGHTS,
    List_of_READ_MATRIX,
    List_of_LIKES,
    List_of_POSTS,
    List_of_GRAPHS,
    List_of_INDIVIDUAL_LIKES) = await run_simulation(PARAMS)

# --- save results (new signature incl. INDIVIDUAL_LIKES) ---
out_root = "runs/Exp_W_success_4"
save_simulation_results(
    out_root,
    List_of_WEIGHTS, List_of_READ_MATRIX, List_of_LIKES,
    List_of_POSTS, List_of_GRAPHS, List_of_INDIVIDUAL_LIKES,
    PARAMS
)
print(f"Saved run to: {out_root}")

In [ ]:
def load_params(path="PARAMS.json"):
    with open(path, "r") as f:
        params = json.load(f)

    # normalize legacy keys
    if "warmup_length" not in params and "warmup_length" in params:
        params["warmup_length"] = params.pop("warmup_length")

    return params


# --- load base params ---
PARAMS = load_params("PARAMS.json")

# --- experiment knobs (same as you used before) ---
PARAMS["time_decay_rate"]    = np.log(2) / 2   # half-life = 2 timesteps
PARAMS["W_agent_success"]    = 0
PARAMS["W_personal_weights"] = 1
PARAMS["W_post_success"]     = 0
PARAMS["noise_level"]        = 0.01

# (optional) quick overrides
PARAMS["replicas"] = 5

PARAMS["num_agents"]   = 30
PARAMS["timesteps"]    = 30
PARAMS["warmup_length"] = 3



# --- secrets from .env (kept out of JSON for safety) ---
ENV = dotenv_values(".env")
for k in ("HF_TOKEN", "API_URL", "MODEL"):
    if k not in ENV or not ENV[k]:
        raise RuntimeError(f"Missing {k} in .env")
    PARAMS[k] = ENV[k]


# --- run simulation ---
(List_of_WEIGHTS1,
    List_of_READ_MATRIX1,
    List_of_LIKES1,
    List_of_POSTS1,
    List_of_GRAPHS1,
    List_of_INDIVIDUAL_LIKES1) = await run_simulation(PARAMS)

# --- save results (new signature incl. INDIVIDUAL_LIKES) ---
out_root = "runs/Exp_W_personal"
save_simulation_results(
    out_root,
    List_of_WEIGHTS1, List_of_READ_MATRIX1, List_of_LIKES1,
    List_of_POSTS1, List_of_GRAPHS1, List_of_INDIVIDUAL_LIKES1,
    PARAMS
)
print(f"Saved run to: {out_root}")

In [ ]:
def load_params(path="PARAMS.json"):
    with open(path, "r") as f:
        params = json.load(f)

    # normalize legacy keys
    if "warmup_length" not in params and "warmup_length" in params:
        params["warmup_length"] = params.pop("warmup_length")

    return params


# --- load base params ---
PARAMS = load_params("PARAMS.json")

# --- experiment knobs (same as you used before) ---
PARAMS["time_decay_rate"]    = np.log(2) / 2   # half-life = 2 timesteps
PARAMS["W_agent_success"]    = 0
PARAMS["W_personal_weights"] = 0
PARAMS["W_post_success"]     = 1
PARAMS["noise_level"]        = 0.01

# (optional) quick overrides
PARAMS["replicas"] = 5

PARAMS["num_agents"]   = 30
PARAMS["timesteps"]    = 30
PARAMS["warmup_length"] = 3



# --- secrets from .env (kept out of JSON for safety) ---
ENV = dotenv_values(".env")
for k in ("HF_TOKEN", "API_URL", "MODEL"):
    if k not in ENV or not ENV[k]:
        raise RuntimeError(f"Missing {k} in .env")
    PARAMS[k] = ENV[k]


# --- run simulation ---
(List_of_WEIGHTS2,
    List_of_READ_MATRIX2,
    List_of_LIKES2,
    List_of_POSTS2,
    List_of_GRAPHS2,
    List_of_INDIVIDUAL_LIKES2) = await run_simulation(PARAMS)

# --- save results (new signature incl. INDIVIDUAL_LIKES) ---
out_root = "runs/Exp_W_post"
save_simulation_results(
    out_root,
    List_of_WEIGHTS2, List_of_READ_MATRIX2, List_of_LIKES2,
    List_of_POSTS2, List_of_GRAPHS2, List_of_INDIVIDUAL_LIKES2,
    PARAMS
)
print(f"Saved run to: {out_root}")

In [ ]:

# Load a specific replica
replica_dir = "runs/r000"  # Replace with your actual path
replica_data = load_replica(replica_dir)

# Extract the individual likes data
individual_likes_data = replica_data["INDIVIDUAL_LIKES"]

# Check its structure
print("Shape:", individual_likes_data.shape)
print("Type:", type(individual_likes_data))

In [ ]:
print("List shape:", List_of_INDIVIDUAL_LIKES[0].shape)
print("Loaded shape:", individual_likes_data.shape)
print("Are they equal?", np.array_equal(List_of_INDIVIDUAL_LIKES[0], individual_likes_data))

In [ ]:
fig = show_interactive_conversation_tree_with_agent_selection("runs/r000")


In [ ]:
print(List_of_POSTS[0][3][0])